# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [56]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys1 import geoapify_key

In [57]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,294.12,95,23,2.79,ZA,1706913570
1,1,waitangi,-43.9535,-176.5597,287.49,95,100,1.79,NZ,1706913701
2,2,lata,40.1629,-8.3327,283.66,84,35,1.65,PT,1706913702
3,3,yellowknife,62.4560,-114.3525,262.75,95,100,2.57,CA,1706913702
4,4,hammerfest,70.6634,23.6821,265.40,93,0,0.51,NO,1706913702


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [74]:
%%capture --no-display

# Configure the map plot
map_plot_1 =city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.01,
    color = "City"
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria and drop any results with null values
# 273.15 kelvins need to be subtracted or aded for Celcius conversion
ideal_vacation_df=city_data_df.loc[(city_data_df["Max Temp"]<=305.15) & (city_data_df["Max Temp"]>=294.15) & (city_data_df["Cloudiness"]==0) & (city_data_df["Wind Speed"]<4.5),:]

# Drop any rows with null values
ideal_vacation_df=ideal_vacation_df.dropna()

# Display sample data
ideal_vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,port augusta,-32.5000,137.7667,301.39,30,0,2.06,AU,1706913893
145,145,port lincoln,-34.7333,135.8667,295.16,67,0,2.26,AU,1706913723
236,236,banikoara,11.2985,2.4386,297.93,14,0,3.39,BJ,1706913744
256,256,crucecita,-28.9500,-58.9833,301.86,55,0,3.45,AR,1706913750
305,305,remire-montjoly,4.9167,-52.2667,298.17,88,0,0.00,GF,1706913756
402,402,muscle shoals,34.7448,-87.6675,295.23,35,0,2.57,US,1706913770
434,434,cruzeiro do sul,-29.5128,-51.9853,297.20,63,0,2.79,BR,1706913774
486,486,katiola,8.1333,-5.1000,300.50,23,0,2.24,CI,1706913781


### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= ideal_vacation_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
31,port augusta,AU,-32.5000,137.7667,30,
145,port lincoln,AU,-34.7333,135.8667,67,
236,banikoara,BJ,11.2985,2.4386,14,
256,crucecita,AR,-28.9500,-58.9833,55,
305,remire-montjoly,GF,4.9167,-52.2667,88,
402,muscle shoals,US,34.7448,-87.6675,35,
434,cruzeiro do sul,BR,-29.5128,-51.9853,63,
486,katiola,CI,8.1333,-5.1000,23,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [62]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "radius": radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame -xpert Learning Assistant
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
#name_address #check if key is being read

Starting hotel search
port augusta - nearest hotel: Hotel Commonwealth
port lincoln - nearest hotel: Boston Hotel
banikoara - nearest hotel: No hotel found
crucecita - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
muscle shoals - nearest hotel: Days Inn by Wyndham Muscle Shoals
cruzeiro do sul - nearest hotel: Estrela Palace Hotel
katiola - nearest hotel: Hôtel Amitie


,City,Country,Lat,Lng,Humidity,Hotel Name
31,port augusta,AU,-32.5000,137.7667,30,Hotel Commonwealth
145,port lincoln,AU,-34.7333,135.8667,67,Boston Hotel
236,banikoara,BJ,11.2985,2.4386,14,No hotel found
256,crucecita,AR,-28.9500,-58.9833,55,No hotel found
305,remire-montjoly,GF,4.9167,-52.2667,88,Complexe Belova
402,muscle shoals,US,34.7448,-87.6675,35,Days Inn by Wyndham Muscle Shoals
434,cruzeiro do sul,BR,-29.5128,-51.9853,63,Estrela Palace Hotel
486,katiola,CI,8.1333,-5.1000,23,Hôtel Amitie


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [75]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.01,
    color = "City",
    hover_cols=['Hotel Name', 'Country']  # Addding to hover message
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)